# Credit card fraud detection
Recent estimates suggest that in 2020, credit card fraud was responsible for losses totaling 28.58 billion dollars ([Nilson, 2021](https://nilsonreport.com/upload/content_promo/NilsonReport_Issue1209.pdf)). The ability to accurately detect fraud protects and ensures customer peace of mind and can prevent massive financial losses. The code in this project helps predict financial fraud by analysing credit card data, which contains typical raw features of credit card transactions such as transaction time, merchant, amount, credit card, and customer details (see [Bahnsen et al., 2016](https://www.sciencedirect.com/science/article/abs/pii/S0957417415008386)

Estimaciones sugieren que, en 2020, el fraude con tarjetas de crédito fue responsable de pérdidas por un total de 28.580 millones de dólares (Nilson, 2021). La capacidad de detectar con precisión el fraude protege y garantiza la tranquilidad de los clientes y puede evitar pérdidas financieras masivas.
Este codigo ayuda a la prediccion de fraudes tomando data de transacciones de tarjetas de credito, como la hora de la transacción, el comercio, el importe, la tarjeta de crédito y los detalles del cliente (véase Bahnsen et al., 2016)(https://www.sciencedirect.com/science/article/abs/pii/S0957417415008386).


## Imports and data preparation


In [10]:
import numpy as np
import pandas as pd
from datetime import date
from geopy import distance

### Load data


In [11]:
# Set path to data
path = "data/fraud_data.csv"

# Specify the transaction time column
trans_time = "trans_date_trans_time"

# Specify any additional date columns
date_cols = ["dob", trans_time]

# Read in the data as a DataFrame and set the index
fraud_df = pd.read_csv(path, parse_dates=date_cols, index_col=trans_time).sort_index()

# Preview the data
fraud_df

,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,customer_country,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,merchant_country,is_fraud
trans_date_trans_time,,,,,,,,,,,,,,,,,,,,,,,
2020-06-21 13:35:01,4822367783500458,Conroy-Emard,food_dining,11.18,Christopher,Farrell,M,97070 Anderson Land,Haines City,FL,33844.0,28.0758,-81.5929,United States,33804.0,Exercise physiologist,1991-01-01,0183bebc44f97bd8ed49a27e7504ad0a,1.371822e+09,28.944803,-81.819858,United States,0.0
2020-06-21 14:02:27,4951647200979051,Kutch-Wilderman,home,18.76,Kimberly,Miller,F,75533 Tamara Valleys,Logan,IL,62856.0,37.9943,-88.9417,United States,324.0,"Scientist, research (physical sciences)",1976-06-15,e364cbb43064585db6fef8adad798ae3,1.371823e+09,38.621529,-88.338930,United States,0.0
2020-06-21 14:12:45,6011693194885790,"Heathcote, Yost and Kertzmann",shopping_net,5.48,Victoria,Fleming,F,2807 Parker Station Suite 080,Stanchfield,MN,55080.0,45.6675,-93.2433,United States,2607.0,"Lecturer, further education",1995-12-04,dccc50d067404415f5546ce81c6ee9e4,1.371824e+09,45.339446,-92.880697,United States,0.0
2020-06-21 15:31:52,675990301623,Hackett Group,travel,6.22,Amanda,Spencer,F,6682 Green Forks,Ogdensburg,NJ,7439.0,41.0767,-74.5982,United States,2456.0,Senior tax professional/tax inspector,1994-03-13,116301dce011c7a0190696720339620d,1.371829e+09,41.041988,-73.652938,United States,0.0
2020-06-21 17:28:16,4128730454058057622,Abernathy and Sons,food_dining,36.04,Monique,Martin,F,68276 Matthew Springs,Ratcliff,TX,75858.0,31.3833,-95.0619,United States,43.0,"Engineer, production",1949-10-04,93079573b9ea6535a5361ad2af1237d8,1.371836e+09,30.669891,-94.919879,United States,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-21 09:30:58,4424338559877976,"Terry, Johns and Bins",misc_pos,37.83,Denise,Barnett,F,23220 Eaton Harbors,Kirby,OH,43330.0,40.8130,-83.4196,United States,118.0,Private music teacher,1957-11-12,54a277240ae5e40df89af08d55a77993,1.379756e+09,41.499281,-83.222959,United States,0.0
2020-09-21 10:48:04,4155021259183870,Huels-Nolan,gas_transport,70.23,Renee,Parrish,F,174 Jennifer Meadow Apt. 467,Mountain Park,OK,73559.0,34.7032,-98.9591,United States,540.0,Research scientist (life sciences),1983-10-12,286ef7d5ac19160c37d42b7d67b5ac53,1.379760e+09,34.175115,-98.646250,United States,0.0
2020-09-21 12:03:46,4822367783500458,Torphy-Kertzmann,health_fitness,11.83,Christopher,Farrell,M,97070 Anderson Land,Haines City,FL,33844.0,28.0758,-81.5929,United States,33804.0,Exercise physiologist,1991-01-01,7b6bac5fd68d47e7410f95901c599d00,1.379765e+09,29.026640,-81.306019,United States,0.0


### Inspect the features and data types
### Inspeccionando tipos de datos en el df

In [12]:
# Print summary of the DataFrame
fraud_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1261 entries, 2020-06-21 13:35:01 to 2020-09-21 12:49:44
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   cc_num            1261 non-null   int64         
 1   merchant          1261 non-null   object        
 2   category          1261 non-null   object        
 3   amt               1261 non-null   float64       
 4   first             1261 non-null   object        
 5   last              1261 non-null   object        
 6   gender            1261 non-null   object        
 7   street            1261 non-null   object        
 8   city              1261 non-null   object        
 9   state             1261 non-null   object        
 10  zip               1261 non-null   float64       
 11  lat               1261 non-null   float64       
 12  long              1261 non-null   float64       
 13  customer_country  1261 non-null   object  

## Customer and Transaction Details
### Extracting age from date of birth
As noted in Bahnsen et al., 2016 (referenced earlier), the customer's age is a common feature in raw credit card data. In the cell below, we add an age column based on the date of birth. 


In [13]:
# Specify the customer date of birth column
dob = "dob"

# Define a function to extract age from date of birth
def age(date_of_birth):
    today = date.today()
    return (
        date.today().year
        - date_of_birth.year
        - ((today.month, today.day) < (date_of_birth.month, date_of_birth.day))
    )


# Create a new column
fraud_df["age"] = fraud_df[dob].apply(age)

### Distance from merchant
The next feature we will create will be the distance between the customer and the merchant. The reasoning for this feature is that the transactions at merchants that are further away from the customer may be a sign of suspicious activity.

This step assumes you have the coordinates (latitude and longitude) of both the customer and the merchant.

In [14]:
# Specify the customer and merchant coordinate columns
customer_lat, customer_long = "lat", "long"
merchant_lat, merchant_long = "merch_lat", "merch_long"

# Create function to calculate the distance between two sets of coordinates
calculate_distance = lambda x: distance.geodesic(
    (x[customer_lat], x[customer_long]), (x[merchant_lat], x[merchant_long])
).km

# Create a new column for the distance
fraud_df["distance_from_merchant"] = fraud_df.apply(calculate_distance, axis=1)

### Domestic transactions
We can also check whether a transaction is domestic or not by comparing the country of the customer and the country of the merchant. 

In [15]:
# Specify the customer country and merchant country
customer_country, merchant_country = "customer_country", "merchant_country"

# Create new column indicating domestic transactions
fraud_df["is_domestic"] = fraud_df[customer_country] == fraud_df[merchant_country]

### Whole number transactions
It can also be useful to determine whether the transaction is a whole number (which may indicate a suspicious transaction).

👇&nbsp;&nbsp;_Make sure to update the transaction amount label below._

In [16]:
# Specify the transaction amount column
amt = "amt"

# Create a column when the transaction amount is a whole number
fraud_df["is_whole_number"] = fraud_df[amt] == fraud_df[amt].astype(int)

### Time and date of transaction
Although the data contains the time of the transaction, it may be useful to extract components of this for predictive purposes. For example, fraud may be more likely at different times of the day or year.

In [17]:
# Extract the hour, day, and month from the datetime index
fraud_df["hour"] = fraud_df.index.hour
fraud_df["day"] = fraud_df.index.day
fraud_df["month"] = fraud_df.index.month

### Preview the new customer and transaction details features
❗&nbsp;&nbsp;_If you omit or add other features, adapt the list of columns below accordingly._

In [18]:
fraud_df[
    [
        "age",
        "distance_from_merchant",
        "is_domestic",
        "is_whole_number",
        "hour",
        "day",
        "month",
    ]
]

,age,distance_from_merchant,is_domestic,is_whole_number,hour,day,month
trans_date_trans_time,,,,,,,
2020-06-21 13:35:01,33,98.839094,True,False,13,21,6
2020-06-21 14:02:27,47,87.331893,True,False,14,21,6
2020-06-21 14:12:45,28,46.178250,True,False,14,21,6
2020-06-21 15:31:52,29,79.551483,True,False,15,21,6
2020-06-21 17:28:16,74,80.249480,True,False,17,21,6
...,...,...,...,...,...,...,...
2020-09-21 09:30:58,66,77.982903,True,False,9,21,9
2020-09-21 10:48:04,40,65.256515,True,False,10,21,9
2020-09-21 12:03:46,33,109.055196,True,False,12,21,9


## Customer History
Customer history is an important indicator of a suspicious transaction. For example, if a customer suddenly increases the volume or cost of their transactions, this may indicate their card has become compromised. The cells below create a number of features that reflect a customer's transaction history.

### Average transaction amount over time
The first general feature we will engineer will be the average amount a customer spends on their credit card.

👇&nbsp;&nbsp;_Make sure to update the credit card number label below._

In [19]:
# Specify the credit card number column
cc_num = "cc_num"

# Calculate the average transaction amount for the credit card across all purchases
fraud_df["avg_amount"] = fraud_df.groupby(cc_num)[amt].transform(np.mean)

### Total number of transactions at a merchant over time
Knowing how often a customer shops at a particular merchant is also helpful. If they have never shopped at a specific merchant, it may indicate that it is not them making the transaction.


In [20]:
# Specify the merchant column
merchant = "merchant"

# Calculate the total number of purchases made at a merchant
fraud_df["merchant_qty"] = fraud_df.groupby([cc_num, merchant])[amt].transform(
    np.size
)

### Average monthly transaction for the past 30 days
Customer spending habits may change over time. This cell calculates the average amount per transaction for the past 30 days.

In [21]:
# Calculate the average amount per credit card for the past 30 days
fraud_monthly_amt = fraud_df.groupby(cc_num)[amt].rolling("30D").mean().reset_index()

# Rename the column
fraud_monthly_amt.rename(columns={amt: "avg_30_day"}, inplace=True)

# Merge in the new column by cc number and transaction time
fraud_df = (
    fraud_df.merge(fraud_monthly_amt, on=[cc_num, trans_time])
    .set_index(trans_time)
    .sort_index()
)

### Total number of transactions with the same merchant over the past 30 days
Like the cell above, customers may discover new shopping places or stop shopping at existing merchants. This cell calculates the total number of transactions at a merchant over the past 30 days.

In [29]:
# Calculate the total number of transactions per credit card for the past 30 days
fraud_monthly_qty = (
    fraud_df.groupby([cc_num, merchant])[amt]
    .rolling("30D")
    .count()
    .reset_index()
    .drop(columns=merchant)
)

# Rename the column
fraud_monthly_qty.rename(columns={amt: "merchant_qty_30_day"}, inplace=True)

# Merge in the new column by cc number and transaction time
fraud_df = (
    fraud_df.merge(fraud_monthly_qty, on=[cc_num, trans_time])
    .set_index(trans_time)
    .sort_index()
)

### Average Daily Transaction Amount for Past 30 Days
This cell provides another form of spending history: how much they spend on average on a daily basis, using data from the past 30 days.

In [30]:
# Calculate the total amount spent per day
fraud_df["date"] = fraud_df.index.strftime("%d-%m-%Y")
fraud_df["day_amt"] = fraud_df.groupby([cc_num, "date"])[amt].transform(np.sum)

# Calculate the average daily amount per credit card for the past 30 days
fraud_daily_avg = (
    fraud_df.groupby([cc_num])["day_amt"].rolling("30D").mean().reset_index()
)

# Rename the column
fraud_daily_avg.rename(columns={"day_amt": "daily_avg_30_day"}, inplace=True)

# Merge in the new column by cc number and transaction time
fraud_df = (
    fraud_df.merge(fraud_daily_avg, on=[cc_num, trans_time])
    .set_index(trans_time)
    .sort_index()
)

### Average amount by category for past 30 days
If a customer is not into health and fitness and suddenly makes many large purchases at a fitness warehouse, it may be a sign of suspicious activity. This cell calculates the average amount spent in a category over the past 30 days.

In [24]:
# Specify the transaction category column
category = "category"

# Calculate the average amount spent in a category for the past 30 days
fraud_category_avg_month = (
    fraud_df.groupby([cc_num, category])[amt]
    .rolling("30D")
    .mean()
    .reset_index()
    .drop(columns=category)
)

# Rename the column
fraud_category_avg_month.rename(columns={amt: "category_avg_month"}, inplace=True)

# Merge in the new column by cc number and transaction time
fraud_df = (
    fraud_df.merge(fraud_category_avg_month, on=[cc_num, trans_time])
    .set_index(trans_time)
    .sort_index()
)

### Total number of transactions on this day up to this transaction
If a credit card is used for fraudulent activity, there may be an increase in the frequency of transactions in a short time. This cell calculates the total number of transactions in the day prior to the current transaction.

In [31]:
# Calculate the total number of transactions in a day up to current transaction
fraud_day_transactions = (
    fraud_df.groupby(cc_num)[amt]
    .rolling("1D", closed="left")
    .count()
    .replace({np.nan: 0})
    .reset_index()
)

# Rename the column
fraud_day_transactions.rename(columns={amt: "day_qty"}, inplace=True)

# Merge in the new column by cc number and transaction time
fraud_df = (
    fraud_df.merge(fraud_day_transactions, on=[cc_num, trans_time])
    .set_index(trans_time)
    .sort_index()
)

### Preview the new customer history features
❗&nbsp;&nbsp;_If you omit or add other features, adapt the list of columns below accordingly._

In [26]:
# Preview new columns
fraud_df[
    [
        amt,
        category,
        merchant,
        "avg_amount",
        "merchant_qty",
        "avg_30_day",
        "merchant_qty_30_day",
        "daily_avg_30_day",
        "category_avg_month",
        "day_qty",
    ]
]

,amt,category,merchant,avg_amount,merchant_qty,avg_30_day,merchant_qty_30_day,daily_avg_30_day,category_avg_month,day_qty
trans_date_trans_time,,,,,,,,,,
2020-06-21 13:35:01,11.18,food_dining,Conroy-Emard,36.516667,1,11.1800,1.0,11.1800,11.18,0.0
2020-06-21 14:02:27,18.76,home,Kutch-Wilderman,13.015000,1,18.7600,1.0,18.7600,18.76,0.0
2020-06-21 14:12:45,5.48,shopping_net,"Heathcote, Yost and Kertzmann",36.360000,1,5.4800,1.0,5.4800,5.48,0.0
2020-06-21 15:31:52,6.22,travel,Hackett Group,15.350000,1,6.2200,1.0,6.2200,6.22,0.0
2020-06-21 17:28:16,36.04,food_dining,Abernathy and Sons,98.365000,1,36.0400,1.0,36.0400,36.04,0.0
...,...,...,...,...,...,...,...,...,...,...
2020-09-21 09:30:58,37.83,misc_pos,"Terry, Johns and Bins",43.495000,1,37.8300,1.0,37.8300,37.83,0.0
2020-09-21 10:48:04,70.23,gas_transport,Huels-Nolan,63.150000,1,69.3200,1.0,69.3200,69.32,0.0
2020-09-21 12:03:46,11.83,health_fitness,Torphy-Kertzmann,36.516667,1,11.8300,1.0,11.8300,11.83,0.0


## Merchant and Category History
Some merchants and categories may be at a higher risk of fraud than others. You can use the following code to create a column based on the proportion of fraud for each merchant and category.

👇&nbsp;&nbsp;_Make sure to update the fraud label below._

In [27]:
# Specify the fraud label column
is_fraud = "is_fraud"

# Calculate the proportion of fraud per merchant
fraud_df["merch_fraud_prop"] = fraud_df.groupby(merchant)[is_fraud].transform("mean")

# Calculate the proportion of fraud per category
fraud_df["category_fraud_prop"] = fraud_df.groupby(category)[is_fraud].transform("mean")

### Preview the new merchant and category history features



In [28]:
fraud_df[[merchant, category, "merch_fraud_prop", "category_fraud_prop"]]

,merchant,category,merch_fraud_prop,category_fraud_prop
trans_date_trans_time,,,,
2020-06-21 13:35:01,Conroy-Emard,food_dining,0.0,0.000000
2020-06-21 14:02:27,Kutch-Wilderman,home,0.0,0.000000
2020-06-21 14:12:45,"Heathcote, Yost and Kertzmann",shopping_net,0.0,0.009901
2020-06-21 15:31:52,Hackett Group,travel,0.0,0.000000
2020-06-21 17:28:16,Abernathy and Sons,food_dining,0.0,0.000000
...,...,...,...,...
2020-09-21 09:30:58,"Terry, Johns and Bins",misc_pos,0.0,0.013333
2020-09-21 10:48:04,Huels-Nolan,gas_transport,0.0,0.000000
2020-09-21 12:03:46,Torphy-Kertzmann,health_fitness,0.0,0.000000
